In [ ]:
#Install packages

! pip install transformers datasets
! pip install torch
! pip install tensorflow
! pip install tika
! pip install nltk

from transformers import pipeline
import os
import re
import math
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('all')

In [7]:
!git clone https://github.com/bcarter24/testRepo.git

Cloning into 'testRepo'...
remote: Enumerating objects: 298, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (288/288), done.
remote: Total 298 (delta 8), reused 295 (delta 8), pack-reused 0
Receiving objects: 100% (298/298), 5.95 MiB | 27.20 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [23]:
%cd /content/testRepo/data
!mkdir tokenizedFiles

'ATF Rulings and Memos'/  'US Federal Code of Law'/
/content/testRepo/data
'ATF Rulings and Memos'/  'US Federal Code of Law'/


In [17]:
#!mkdir tokenizedFiles
%ls

Code/  data/  LICENSE  tokenizedFiles/
